In [6]:
!pip install statsmodels

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import qgrid
from numpy.random import normal, seed

from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA

import math

from sklearn.metrics import mean_squared_error


ModuleNotFoundError: No module named 'statsmodels'

In [8]:
import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)
    
else:
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False

# 전처리

## 전처리하기

In [9]:
data = pd.read_csv('./data.csv', encoding="CP949")

In [10]:
data['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   구분      368088 non-null  int64         
 1   공급량     368088 non-null  float64       
 2   time    368088 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 8.4 MB


In [11]:
d_map = {}
for i, d in enumerate(data['구분'].unique()):
    d_map[d] = i
    
data['구분'] = data['구분'].map(d_map)

In [12]:
data['연월일'] = pd.to_datetime(data['연월일'])

In [13]:
data['time'] = data.apply(lambda x: x['연월일'] + timedelta(hours=x['시간']), axis=1)

In [14]:
data

,연월일,시간,구분,공급량,time
0,2013-01-01,1,0,2497.129,2013-01-01 01:00:00
1,2013-01-01,2,0,2363.265,2013-01-01 02:00:00
2,2013-01-01,3,0,2258.505,2013-01-01 03:00:00
3,2013-01-01,4,0,2243.969,2013-01-01 04:00:00
4,2013-01-01,5,0,2344.105,2013-01-01 05:00:00
...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018-12-31 20:00:00
368084,2018-12-31,21,6,669.961,2018-12-31 21:00:00
368085,2018-12-31,22,6,657.941,2018-12-31 22:00:00
368086,2018-12-31,23,6,610.953,2018-12-31 23:00:00


In [15]:
data = data.drop(['연월일', '시간'], axis=1)

In [16]:
data

,구분,공급량,time
0,0,2497.129,2013-01-01 01:00:00
1,0,2363.265,2013-01-01 02:00:00
2,0,2258.505,2013-01-01 03:00:00
3,0,2243.969,2013-01-01 04:00:00
4,0,2344.105,2013-01-01 05:00:00
...,...,...,...
368083,6,681.033,2018-12-31 20:00:00
368084,6,669.961,2018-12-31 21:00:00
368085,6,657.941,2018-12-31 22:00:00
368086,6,610.953,2018-12-31 23:00:00


### 데이터 정규화

In [24]:
from sklearn.preprocessing import MinMaxScaler

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   구분      368088 non-null  int64         
 1   공급량     368088 non-null  float64       
 2   time    368088 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 8.4 MB


In [26]:
scaler = MinMaxScaler()
scaled_cols = ['공급량']
data_scaled = scaler.fit_transform(data[scaled_cols])
data['scaled_공급량'] = data_scaled

## train, val 나누기

In [27]:
train_year = [2013, 2014, 2015, 2016, 2017]
val_year = [2018]

In [28]:
train = data[data['time'].dt.year.isin(train_year)]
val = data[data['time'].dt.year.isin(val_year)]

In [29]:
features = ['time']
train_x = train[features]
train_y = train['scaled_공급량']

val_x = val[features]
val_y = val['scaled_공급량']

In [30]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

((306761, 1), (306761,), (61320, 1), (61320,))

In [31]:
plt.figure(figsize=(12, 6))
sns.violinplot(x='time', y='scaled_공급량', data=data)

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x0000024B62BC4EE0> (for post_execute):


KeyboardInterrupt: 

# EDA

# 학습

In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model


from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequentail()
model.add(LSTM(16,
              input_shape=(train_x.shape[1], train_x.shape[2]),
              activation='relu',
              return_squences=False)
         )
model.add(Dense(1))

AlreadyExistsError: Another metric with the same name already exists.

In [134]:
import lightgbm as lgb

In [135]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective':'regression',
    'metric':'mae',
    'seed':42
}


In [136]:
model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

C:\Users\SeunghaLee\.conda\envs\data\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\SeunghaLee\.conda\envs\data\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 244.857
[40]	valid_0's l1: 174.855
[60]	valid_0's l1: 158.739
[80]	valid_0's l1: 153.323
[100]	valid_0's l1: 150.948
[120]	valid_0's l1: 150.463
Early stopping, best iteration is:
[112]	valid_0's l1: 150.297


# 추론 및 결과 제출

In [22]:
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')

In [23]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [24]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [27]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [29]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [30]:
test['구분'] = test['구분'].map(d_map)

In [31]:
test_x = test[features]

In [32]:
test_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [33]:
preds = model.predict(test_x)

In [34]:
submission['공급량'] = preds

In [38]:
submission.to_csv('baseline.csv', index=False)